In [1]:
import numpy as np
import pandas as pd
import nltk
# import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     zhangdce/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jupyter-
[nltk_data]     zhangdce/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Load data into dataframe
df = pd.read_csv('~/Customer_Reviews/Data/watch_reviews.tsv', sep='\t', error_bad_lines=False)

/tmp/ipykernel_74098/1500074030.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('~/Customer_Reviews/Data/watch_reviews.tsv', sep='\t', error_bad_lines=False)
Skipping line 8704: expected 15 fields, saw 22
Skipping line 16933: expected 15 fields, saw 22
Skipping line 23726: expected 15 fields, saw 22

Skipping line 85637: expected 15 fields, saw 22

Skipping line 132136: expected 15 fields, saw 22
Skipping line 158070: expected 15 fields, saw 22
Skipping line 166007: expected 15 fields, saw 22
Skipping line 171877: expected 15 fields, saw 22
Skipping line 177756: expected 15 fields, saw 22
Skipping line 181773: expected 15 fields, saw 22
Skipping line 191085: expected 15 fields, saw 22
Skipping line 196273: expected 15 fields, saw 22
Skipping line 196331: expected 15 fields, saw 22

Skipping line 197000: expected 15 fields, saw 22
Skipping line 197011: expected 15 fields, 

In [4]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0.0,0.0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0.0,0.0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1.0,1.0,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0.0,0.0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0.0,0.0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [5]:
df.shape

(582324, 15)

In [6]:
# Remove missing value
df.dropna(subset=['review_body'],inplace=True)

In [7]:
df.reset_index(inplace=True, drop=True)

In [8]:
df.shape

(582175, 15)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582175 entries, 0 to 582174
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   marketplace        582175 non-null  object 
 1   customer_id        582175 non-null  int64  
 2   review_id          582175 non-null  object 
 3   product_id         582175 non-null  object 
 4   product_parent     582175 non-null  int64  
 5   product_title      582174 non-null  object 
 6   product_category   582175 non-null  object 
 7   star_rating        582175 non-null  int64  
 8   helpful_votes      582175 non-null  float64
 9   total_votes        582175 non-null  float64
 10  vine               582175 non-null  object 
 11  verified_purchase  582175 non-null  object 
 12  review_headline    582169 non-null  object 
 13  review_body        582175 non-null  object 
 14  review_date        582173 non-null  object 
dtypes: float64(2), int64(3), object(10)
memory usage: 6

In [10]:
# use the first 1000 data as our training data
data = df.loc[:999, 'review_body'].tolist()

In [11]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english') #stopwords.append("n't")
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("br") #html <br>
stopwords.append("watch")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 183 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [12]:
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [13]:
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'wear',
 'dainti']

In [14]:
# TF-IDF

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

/opt/tljh/user/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/tljh/user/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.

In total, there are 1000 reviews and 239 terms.


In [16]:
tfidf_matrix

<1000x239 sparse matrix of type '<class 'numpy.float64'>'
	with 6891 stored elements in Compressed Sparse Row format>

In [19]:
tfidf_matrix.toarray() 

array([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [20]:
tfidf_matrix.todense()

matrix([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]])

In [21]:
print(type(tfidf_matrix.toarray()))

<class 'numpy.ndarray'>


In [22]:
print(type(tfidf_matrix.todense()))

<class 'numpy.matrix'>


In [24]:
# K means 
# 缺点：分类不平均，有些分类比较大，有些比较小

In [25]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [26]:
# create DataFrame films from all of the input files.
product = { 'review': df[:1000].review_body, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [27]:
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,3
1,I love this watch it keeps time wonderfully.,3
2,Scratches,0
3,"It works well on me. However, I found cheaper ...",0
4,Beautiful watch face. The band looks nice all...,0
5,"i love this watch for my purpose, about the pe...",3
6,"for my wife and she loved it, looks great and ...",2
7,I was about to buy this thinking it was a Swis...,0
8,Watch is perfect. Rugged with the metal &#34;B...,2
9,Great quality and build.<br />The motors are r...,0


In [28]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
0,654
3,112
2,97
4,75
1,62


In [29]:
km.cluster_centers_

array([[0.00576886, 0.00445864, 0.00390482, ..., 0.0066609 , 0.01779139,
        0.01349022],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00784247,
        0.        ],
       [0.00332465, 0.        , 0.        , ..., 0.00229765, 0.00388971,
        0.02402288],
       [0.        , 0.04089119, 0.        , ..., 0.01211246, 0.01702212,
        0.0039668 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00874405,
        0.        ]])

In [30]:
km.cluster_centers_.shape

(5, 239)

In [47]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [48]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.22462475 0.59097543 0.06109259 0.06157867 0.06172856]
 [0.08403373 0.66166568 0.08561107 0.08352571 0.08516381]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.59993351 0.10000037 0.10000012 0.10000012 0.10006589]
 [0.39245124 0.40562246 0.06727483 0.0677745  0.06687697]
 [0.06723513 0.06846394 0.06953883 0.72747216 0.06728994]]


In [49]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 239)
[[ 0.87548243  7.68286245  0.20018688 ...  0.20076762  0.20032699
   0.20109792]
 [ 0.21041924  0.20229896  0.20194174 ...  3.07689642  1.02191077
   0.20296593]
 [ 0.20400423  0.20151238  2.61888164 ...  0.29474912  0.20332981
   6.22448595]
 [ 1.62766618  0.20421732  0.20068498 ...  0.2022291  12.48214931
   0.20130443]
 [ 2.177755    0.20487172  0.33205984 ...  3.16105503  2.15367091
   5.76725122]]


In [50]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]
print(topic_names)

['Topic0', 'Topic1', 'Topic2', 'Topic3', 'Topic4']


In [51]:
# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

In [52]:
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.22,0.59,0.06,0.06,0.06,1
Doc1,0.08,0.66,0.09,0.08,0.09,1
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.06,0.06,0.06,0.06,0.75,4
Doc4,0.04,0.04,0.04,0.42,0.46,4
Doc5,0.07,0.69,0.07,0.08,0.08,1
Doc6,0.06,0.07,0.49,0.07,0.31,2
Doc7,0.06,0.06,0.06,0.06,0.75,4
Doc8,0.04,0.05,0.05,0.27,0.59,4
Doc9,0.06,0.06,0.77,0.06,0.06,2


In [53]:
df_document_topic['topic'].value_counts().to_frame()

,topic
4,265
2,214
3,198
1,191
0,132


In [54]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[ 0.87548243  7.68286245  0.20018688 ...  0.20076762  0.20032699
   0.20109792]
 [ 0.21041924  0.20229896  0.20194174 ...  3.07689642  1.02191077
   0.20296593]
 [ 0.20400423  0.20151238  2.61888164 ...  0.29474912  0.20332981
   6.22448595]
 [ 1.62766618  0.20421732  0.20068498 ...  0.2022291  12.48214931
   0.20130443]
 [ 2.177755    0.20487172  0.33205984 ...  3.16105503  2.15367091
   5.76725122]]


AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'

In [ ]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words